In [1]:
!pip install pulp -q
from pulp import*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 18.4 MB/s eta 0:00:00


### 1)
Considere uma bonelaria que fabrica dois tipos de produtos (bonés e camisas). Atualmente, o engenheiro de produção responsável elaborou as previsões das próximas semanas (assumindo como conhecidas), descritas na Tabela 1, além de outras informações importantes. Os produtos são similares, e, portanto, a matéria-prima usada é a mesma, mas em quantidades diferentes. Atualmente, devido a pandemia, apenas um centro de trabalho segue em atividade, havendo uma disponibilidade de 44 horas no período, com a produção de uma unidade do boné consumindo 15 minutos do tempo disponível e para as camisas um tempo de processamento que leva 20 minutos, com os custos variando em função da semana. É importante enfatizar que a produção pode ser estocada para os períodos posteriores, mesmo havendo um custo unitário de estoque, podendo ser vantajoso ou não. Sabendo que o engenheiro de produção quer atender a demanda, ao menor custo unitário, qual o plano de produção para o problema? Admita que os estoques iniciais são nulos e ao final do período mencionado, os mesmos se mantenham nulos. Implemente no Excel ou Colab para achar a solução ótima.

---
| Produto     | Informações gerais       | Semana 1 | Semana 2 | Semana 3 |
|-------------|--------------------------|----------|----------|----------|
| **Boné**    | Demanda                 | 90       | 100      | 130      |
|             | Custos de produção   | 20       | 20       | 30       |
|             | Custo para armazenar  | 2        | 3        | -        |
| **Camisas** | Demanda                  | 40       | 60       | 80       |
|             | Custos de produção    | 20       | 20       | 30       |
|             | Custo para armazenar  | 2.5      | 3.5      | -        |


In [2]:

# Problema de otimização
PRO = LpProblem("Plano_de_Producao", LpMinimize)

# Variáveis de produção e estoque
Pb1, Pb2, Pb3 = LpVariable("Pb1", 0), LpVariable("Pb2", 0), LpVariable("Pb3", 0)
Pc1, Pc2, Pc3 = LpVariable("Pc1", 0), LpVariable("Pc2", 0), LpVariable("Pc3", 0)
Ab1, Ab2 = LpVariable("Ab1", 0), LpVariable("Ab2", 0)
Ac1, Ac2 = LpVariable("Ac1", 0), LpVariable("Ac2", 0)

# Função objetivo: Minimizar custos (produção + armazenagem)
PRO += (
    20 * Pb1 + 20 * Pb2 + 30 * Pb3 +
    20 * Pc1 + 20 * Pc2 + 30 * Pc3 +
    2 * Ab1 + 3 * Ab2 +
    2.5 * Ac1 + 3.5 * Ac2
)

# Restrições de tempo disponível
PRO += 15 * Pb1 + 20 * Pc1 <= 2640, "Tempo_Semana_1"
PRO += 15 * Pb2 + 20 * Pc2 <= 2640, "Tempo_Semana_2"
PRO += 15 * Pb3 + 20 * Pc3 <= 2640, "Tempo_Semana_3"

# Restrições de demanda - Bonés
PRO += Pb1 + Ab1 >= 90, "Demanda_Bone_Semana_1"
PRO += Pb2 + Ab2 + Ab1 >= 100, "Demanda_Bone_Semana_2"
PRO += Pb3 + Ab2 >= 130, "Demanda_Bone_Semana_3"

# Restrições de demanda - Camisas
PRO += Pc1 + Ac1 >= 40, "Demanda_Camisa_Semana_1"
PRO += Pc2 + Ac2 + Ac1 >= 60, "Demanda_Camisa_Semana_2"
PRO += Pc3 + Ac2 >= 80, "Demanda_Camisa_Semana_3"

# Estoques iniciais e finais devem ser zero
PRO += Ab2 == 0, "Estoque_Final_Bone"
PRO += Ac2 == 0, "Estoque_Final_Camisa"

# Estoques iniciais são zero
PRO += Ab1 == Pb1 - 90, "Estoque_Bone_Semana_1"
PRO += Ab2 == Pb2 + Ab1 - 100, "Estoque_Bone_Semana_2"
PRO += Ac1 == Pc1 - 40, "Estoque_Camisa_Semana_1"
PRO += Ac2 == Pc2 + Ac1 - 60, "Estoque_Camisa_Semana_2"

# Resolver o problema
PRO.solve()

# Resultado formatado
print("Plano de Produção e Estoque:")
print("-" * 50)

# Resultados de produção
print("Produção semanal:")
print(f"  Bonés: Semana 1 = {Pb1.varValue}, Semana 2 = {Pb2.varValue}, Semana 3 = {Pb3.varValue}")
print(f"  Camisas: Semana 1 = {Pc1.varValue}, Semana 2 = {Pc2.varValue}, Semana 3 = {Pc3.varValue}")

# Estoques intermediários
print("\nEstoque intermediário ao final de cada semana:")
print(f"  Bonés: Semana 1 = {Ab1.varValue}, Semana 2 = {Ab2.varValue}")
print(f"  Camisas: Semana 1 = {Ac1.varValue}, Semana 2 = {Ac2.varValue}")

# Custo total
print("\nCusto total:")
print(f"  Custo de produção e armazenamento: R$ {value(PRO.objective):.2f}")

Plano de Produção e Estoque:
--------------------------------------------------
Produção semanal:
  Bonés: Semana 1 = 90.0, Semana 2 = 100.0, Semana 3 = 130.0
  Camisas: Semana 1 = 40.0, Semana 2 = 60.0, Semana 3 = 80.0

Estoque intermediário ao final de cada semana:
  Bonés: Semana 1 = 0.0, Semana 2 = 0.0
  Camisas: Semana 1 = 0.0, Semana 2 = 0.0

Custo total:
  Custo de produção e armazenamento: R$ 12100.00


### 2)

Dois produtos são fabricados em sequência em duas máquinas. O tempo disponível em cada máquina é de oito horas por dia e pode ser aumentado com quatro horas extras, se necessário, a um custo adicional de R$ 100 por hora. A tabela abaixo fornece a taxa de produção para as duas máquinas, bem como o preço por unidade de dois produtos. A) Escreva o modelo. B) Implemente em planilha
eletrônica. C) Realize uma análise de sensibilidade, incrementando um aumento no preço dos
produtos 1 e 2, em 30% e depois para 40%, e identifique o preço sombra e o custo reduzido. Discorra
sobre os resultados.

---
#### Tabela: Taxa de Produção (unidades/hora)

| Máquina       | Produto 1  | Produto 2  |
|---------------|-----------|-----------|
| **Máquina 1** | 5  | 5   |
| **Máquina 2** | 8   | 4   |
| **Preço por unidade (R$)** | 110 | 118 |


In [3]:

def resolve(producao_dia=0, aumento_preco=0):
    # Criação do problema
    problem = LpProblem("Maximizar_Lucro", LpMaximize)

    # Variáveis de decisão
    x1 = LpVariable("Produto_1", lowBound=0, cat="Continuous")  # Produto 1 unidade/hora
    x2 = LpVariable("Produto_2", lowBound=0, cat="Continuous")  # Produto 2 unidade/hora
    Te1 = LpVariable("Tempo_extra_M1", lowBound=0, upBound=4, cat="Continuous")  # Tempo extra Máquina 1 hora
    Te2 = LpVariable("Tempo_extra_M2", lowBound=0, upBound=4, cat="Continuous")  # Tempo extra Máquina 2 hora

    # Função objetiva principal
    if aumento_preco == 1: # 30% de ajuste nos produtos 1 e 2
        problem += 143 * x1 + 153.4 * x2 - 100 * (Te1 + Te2), "Maximo_de_lucro"
    elif aumento_preco == 2: # 40% de ajuste nos produtos 1 e 2
        problem += 154 * x1 + 165.2 * x2 - 100 * (Te1 + Te2), "Maximo_de_lucro"
    else: # função objetivo padrão
        problem += 110 * x1 + 118 * x2 - 100 * (Te1 + Te2), "Maximo_de_lucro"

    # Restrições
    problem += (x1 + x2) / 5 <= 8 + Te1, "Restrição_M1"
    problem += (x1 + 2 * x2) / 8 <= 8 + Te2, "Restrição_M2"
    problem += Te1 <= 4, "Restrição_M1_Extra"
    problem += Te2 <= 4, "Restrição_M2_Extra"

    # Variável para estimar a produção diária, explorar os melhores cenários de acordo com as restrições
    if producao_dia > 0:
        problem += x1 + x2 <= producao_dia, "Restrição_de_Tolerância"

    # Resolução
    problem.solve()

    # Verifica a solução
    if problem.status == 1:
        precos_sombra = {name: constraint.pi for name, constraint in problem.constraints.items()}
        custos_reduzidos = {var.name: var.dj for var in problem.variables()}

        return {
            "Produto_1": x1.varValue,
            "Produto_2": x2.varValue,
            "Tempo_extra_M1": Te1.varValue,
            "Tempo_extra_M2": Te2.varValue,
            "Lucro_Total": value(problem.objective),
            "Precos_Sombra": precos_sombra,
            "Custo_Reduzido": custos_reduzidos
        }
    else:
        return None


producao_dia = 100  # Estimativa máxima de produção diária unidade/hora

for aumento, perc in zip([0, 1, 2], ["Padrão", "30%", "40%"]):
    print(f"\n--- Cenário de aumento de preços: {perc} ---")
    solucao = resolve(producao_dia=producao_dia, aumento_preco=aumento)

    # Impressão da solução
    if solucao:
        print(f"Quantidade ideal do Produto 1: {solucao['Produto_1']:.2f} unidades/hora")
        print(f"Quantidade ideal do Produto 2: {solucao['Produto_2']:.2f} unidades/hora")
        print(f"Tempo extra necessário na Máquina 1: {solucao['Tempo_extra_M1']:.2f} horas")
        print(f"Tempo extra necessário na Máquina 2: {solucao['Tempo_extra_M2']:.2f} horas")
        print(f"Lucro Total: R$ {solucao['Lucro_Total']:.2f}")

        # Preço Sombra
        print("\nPreços Sombra:")
        for restricao, preco in solucao["Precos_Sombra"].items():
            print(f"{restricao}: {preco:.2f}")

        # Custo Reduzido
        print("\nCustos Reduzidos:")
        for variavel, custo in solucao["Custo_Reduzido"].items():
            print(f"{variavel}: {custo:.2f}")

    else:
        print("Não foi possível encontrar uma solução.")



--- Cenário de aumento de preços: Padrão ---
Quantidade ideal do Produto 1: 56.00 unidades/hora
Quantidade ideal do Produto 2: 4.00 unidades/hora
Tempo extra necessário na Máquina 1: 4.00 horas
Tempo extra necessário na Máquina 2: 0.00 horas
Lucro Total: R$ 6232.00

Preços Sombra:
Restrição_M1: 510.00
Restrição_M2: 64.00
Restrição_M1_Extra: -0.00
Restrição_M2_Extra: -0.00
Restrição_de_Tolerância: -0.00

Custos Reduzidos:
Produto_1: -0.00
Produto_2: -0.00
Tempo_extra_M1: 410.00
Tempo_extra_M2: -36.00

--- Cenário de aumento de preços: 30% ---
Quantidade ideal do Produto 1: 56.00 unidades/hora
Quantidade ideal do Produto 2: 4.00 unidades/hora
Tempo extra necessário na Máquina 1: 4.00 horas
Tempo extra necessário na Máquina 2: 0.00 horas
Lucro Total: R$ 8221.60

Preços Sombra:
Restrição_M1: 663.00
Restrição_M2: 83.20
Restrição_M1_Extra: -0.00
Restrição_M2_Extra: -0.00
Restrição_de_Tolerância: -0.00

Custos Reduzidos:
Produto_1: 0.00
Produto_2: 0.00
Tempo_extra_M1: 563.00
Tempo_extra_M2: 

Analisando os resultados, foi possível perceber que, nos três cenários, embora a quantidade de unidades permaneça constante, o lucro foi aumentando conforme o ajuste nos preços. A Máquina 1 possui restrições de capacidade mais críticas, tornando-se mais valiosa com o aumento dos preços, evidenciado pelos preços sombra crescentes para essa restrição. No entanto, o tempo extra necessário nas Máquinas 1 e 2 não é totalmente aproveitado, com a Máquina 2 especialmente não justificando investimento adicional em tempo extra, dado os custos reduzidos negativos. Por outro lado, a Máquina 1 se beneficiaria de mais tempo extra, o que se reflete nos custos reduzidos positivos para essa variável, sugerindo que o aumento de capacidade nesta máquina traria retorno financeiro. Por fim, para maximizar os lucros, seria interessante investir mais no tempo extra da Máquina 1, especialmente com o aumento nos preços dos produtos.

## 3)
A Castanha S.A é uma empresa que processa castanhas de caju. Atualmente a empresa tem um portfólio de quatro produtos derivados: castanhas assadas com caramelo (CAT001), castanhas caramelizadas sabor café (CATC002), castanhas caramelizadas quebradas (CAT0034) e castanhas tradicionais inteiras (CAT007). A demanda pelo produto vem crescendo significativamente, nos últimos meses, havendo dificuldades para atender os pedidos. Fatores como preço de insumos e o surgimento da concorrência obrigam a empresa a se planejar para ser eficiente no mercado. A programação do mês seguinte já foi enviada ao setor de produção, no qual foram definidos que deve haver uma produção mínima de 1.000 quilos do produto CAT001, entre 300 e 600 quilos do CATC002, no máximo 150 quilos de CAT0034 e não mais que 200 quilos de CAT007. Cada quilo dos produtos CAT001, CATC002, CAT0034 e CAT007 contém, respetivamente 60%, 40%, 20% e 100% de castanhas de caju, com o peso restante correspondente ao caramelo. No momento, existe uma disponibilidade de 1.100 quilos de castanha e 800 quilos de caramelo. Os diversos produtos são fabricados usando quatro tipos de maquinas que processam as castanhas (descascamento, torração, cobertura e embalagem). A tabela seguinte mostra o tempo de processamento por produto em cada máquina. Cada máquina tem 60 horas de tempo disponível no próximo mês. O preço de venda e o custo variável associado a cada quilo de produto é apresentado também.
#A) Modele o problema.
#B)Resolva no Excel ou Colab
#C) Identifique o preço sombra e aplique uma analise de sensibilidade para três cenários diferentes.

### Minutos Necessários por kg

| Máquina      | CAT001 | CAT002 | CAT0034 | CAT007 |
|--------------|--------|--------|---------|--------|
| **Descascamento** | 1      | 1      | 1       | 1      |
| **Torra**        | 2      | 1.6    | 1       | 1.75   |
| **Cobertura**    | 1      | 0.85   | 0.25    | 0      |
| **Embalagem**    | 2.5    | 1.66   | 1.25    | 1      |

---

### Receita e Custo por kg

| Processo        | Descascamento | Torra  | Cobertura | Embalagem |
|-----------------|---------------|--------|-----------|-----------|
| **Preço de venda** | 5.00          | 4.20   | 3.22      | 4.51      |
| **Custo variável** | 3.16          | 2.65   | 2.14      | 3.10      |


In [4]:
# Definição da função com parâmetros ajustáveis
def resolver_problema_ajustado(castanhas=1100, caramelo=800, embalagem=60, max_cat007=200):
    from pulp import LpProblem, LpMaximize, LpVariable

    # Definindo o problema
    problema = LpProblem("Maximizar_Lucro", LpMaximize)

    # Variáveis de decisão
    x1 = LpVariable("CAT001", lowBound=0, cat="Continuous")
    x2 = LpVariable("CAT002", lowBound=0, cat="Continuous")
    x3 = LpVariable("CAT0034", lowBound=0, cat="Continuous")
    x4 = LpVariable("CAT007", lowBound=0, cat="Continuous")

    # Função objetivo
    problema += (
        (5 - 3.16) * x1 + (4.20 - 2.65) * x2 + (3.22 - 2.14) * x3 + (4.51 - 3.10) * x4,
        "Lucro_Total"
    )

    # Restrições de disponibilidade
    problema += 0.6 * x1 + 0.4 * x2 + 0.2 * x3 + 1 * x4 <= castanhas, "Castanhas_Disponiveis"
    problema += 0.4 * x1 + 0.6 * x2 + 0.8 * x3 + 0 * x4 <= caramelo, "Caramelo_Disponivel"

    # Restrição de tempo em máquinas
    problema += (1 * x1 + 1 * x2 + 1 * x3 + 1 * x4) / 60 <= 60, "Descascamento"
    problema += (2 * x1 + 1.6 * x2 + 1 * x3 + 1.75 * x4) / 60 <= 60, "Torra"
    problema += (1 * x1 + 0.85 * x2 + 0.25 * x3) / 60 <= 60, "Cobertura"
    problema += (2.5 * x1 + 1.66 * x2 + 1.25 * x3 + 1 * x4) / 60 <= embalagem, "Embalagem"

    # Restrições de produção mínima e máxima por produto
    problema += x1 >= 1000, "Min_Cat001"
    problema += x2 >= 300, "Min_Cat002"
    problema += x2 <= 600, "Max_Cat002"
    problema += x3 <= 150, "Max_Cat0034"
    problema += x4 <= max_cat007, "Max_Cat007"

    # Resolvendo o problema
    problema.solve()

    # Resultado
    resultado = {
        "CAT001": x1.varValue,
        "CAT002": x2.varValue,
        "CAT0034": x3.varValue,
        "CAT007": x4.varValue,
        "Lucro_Total": problema.objective.value(),
        "Analise_Sensibilidade": {
            nome: {"Preco_Sombra": c.pi, "Margem_Folga": c.slack}
            for nome, c in problema.constraints.items()
        }
    }
    return resultado

# Executando os cenários ajustados
cenarios_ajustados = {
    "Cenário 1 (Sem Alterações)": resolver_problema_ajustado(),
    "Cenário 2 (Aumento na Embalagem)": resolver_problema_ajustado(embalagem=70),
    "Cenário 3 (Aumento no Max_Cat007)": resolver_problema_ajustado(max_cat007=250),
    "Cenário 4 (Alteração de Castanhas e Caramelo)": resolver_problema_ajustado(castanhas=1200, caramelo=850),
}

# Exibindo resultados
for nome, resultado in cenarios_ajustados.items():
    print(f"\n--- {nome} ---")
    for produto, valor in resultado.items():
        if produto != "Analise_Sensibilidade":
            print(f"{produto}: {valor:.2f} kg" if produto != "Lucro_Total" else f"{produto}: R${valor:.2f}")
    print("\nAnálise de Sensibilidade:")
    for nome_restricao, analise in resultado["Analise_Sensibilidade"].items():
        print(f"{nome_restricao}: Preço-sombra = {analise['Preco_Sombra']:.2f}, Margem de folga = {analise['Margem_Folga']:.2f}")





--- Cenário 1 (Sem Alterações) ---
CAT001: 1000.00 kg
CAT002: 542.17 kg
CAT0034: 0.00 kg
CAT007: 200.00 kg
Lucro_Total: R$2962.36

Análise de Sensibilidade:
Castanhas_Disponiveis: Preço-sombra = -0.00, Margem de folga = 83.13
Caramelo_Disponivel: Preço-sombra = -0.00, Margem de folga = 74.70
Descascamento: Preço-sombra = -0.00, Margem de folga = 30.96
Torra: Preço-sombra = -0.00, Margem de folga = 6.38
Cobertura: Preço-sombra = -0.00, Margem de folga = 35.65
Embalagem: Preço-sombra = 56.02, Margem de folga = -0.00
Min_Cat001: Preço-sombra = -0.49, Margem de folga = -0.00
Min_Cat002: Preço-sombra = -0.00, Margem de folga = -242.17
Max_Cat002: Preço-sombra = -0.00, Margem de folga = 57.83
Max_Cat0034: Preço-sombra = -0.00, Margem de folga = 150.00
Max_Cat007: Preço-sombra = 0.48, Margem de folga = -0.00

--- Cenário 2 (Aumento na Embalagem) ---
CAT001: 1339.62 kg
CAT002: 440.25 kg
CAT0034: 0.00 kg
CAT007: 120.13 kg
Lucro_Total: R$3316.67

Análise de Sensibilidade:
Castanhas_Disponiveis:

# Cenário 1 (Sem Alterações)

* Lucro Total: R$2.962,36

* Produção: CAT001 no limite mínimo e CAT007 no limite máximo.
* Gargalos: Embalagem foi o principal gargalo, com preço-sombra de R$56,02 indicando que ampliar a capacidade de embalagem poderia aumentar o lucro.

# Cenário 2 (Aumento na Embalagem)

* Lucro Total: R$3.316,67 (maior aumento entre os cenários).

* Produção: Expansão significativa na produção de CAT001 e CAT002.
* Gargalos: Castanhas e caramelo tornaram-se os novos gargalos, com preços-sombra de R$1,09 e R$0,98, respectivamente. O aumento da embalagem teve um impacto direto no lucro.

# Cenário 3 (Aumento no Max_Cat007)

* Lucro Total: R$2.986,17 (pequeno aumento em relação ao Cenário 1).

* Produção: Maior quantidade de CAT007 (250 kg), mas sem impacto significativo no lucro devido à limitação da embalagem, que continuou como o principal gargalo.
* Gargalos: A embalagem manteve um preço-sombra elevado (R$56,02), limitando maiores ganhos.

# Cenário 4 (Alteração de Castanhas e Caramelo)

* Lucro Total: R$2.962,36 (mesmo do Cenário 1).

* Produção: Não houve mudanças na produção, já que as quantidades de castanhas e caramelo disponíveis no cenário original eram suficientes.
* Gargalos: Embalagem permaneceu como o principal fator limitante, reforçando que alterações nos insumos não afetam significativamente o lucro sem modificações na capacidade de embalagem.

